In [1]:
import sys
sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin

import utils.paramUtil as paramUtil
from options.train_options import TrainCompOptions
from utils.plot_script import *

from models import MotionTransformer
from trainers import DDPMTrainer
from datasets import Text2MotionDataset

from mmcv.runner import get_dist_info, init_dist
from mmcv.parallel import MMDistributedDataParallel, MMDataParallel
import torch
import torch.distributed as dist

### Options:
1. If wanna test full model set opt.debug=False but need to comment out the UnifyLog<br>

In [2]:
class Temp:
    def __init__(self):
        print('mock:: opt')
opt = Temp()

def build_models(opt, dim_pose):
    encoder = MotionTransformer(
        input_feats=dim_pose,
        num_frames=opt.max_motion_length,
        num_layers=opt.num_layers,
        latent_dim=opt.latent_dim,
        no_clip=opt.no_clip,
        no_eff=opt.no_eff)
    return encoder

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/train_options.py
opt.num_layers = 8
opt.latent_dim = 512
opt.diffusion_steps = 1000
opt.no_clip = False
opt.no_eff = False
opt.num_epochs = 50
opt.lr = 2e-4
opt.batch_size = 32
opt.times = 1
opt.feat_bias = 1
opt.is_continue = False
opt.log_every = 50
opt.save_every_e = 5
opt.log_every = 5
opt.eval_every_e = 5
opt.save_latest = 500
opt.is_train = True

# mock from cmd
opt.name = 'TEMP'
opt.batch_size = 128
opt.times = 50
opt.num_epochs = 50
opt.dataset_name = 't2m'
opt.num_layers = 8
opt.diffusion_steps = 1000
opt.data_parallel = True
opt.gpu_id = [0,1,2,3]
opt.device = torch.device("cuda")

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/base_options.py
opt.checkpoints_dir = '../checkpoints'
opt.distributed = False

opt.save_root = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
opt.model_dir = pjoin(opt.save_root, 'model')
opt.meta_dir = pjoin(opt.save_root, 'meta')
opt.data_root = '../data/HumanML3D' # change root to '../'
opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
opt.text_dir = pjoin(opt.data_root, 'texts')
opt.joints_num = 22
opt.debug = True
radius = 4
fps = 20
opt.max_motion_length = 196
dim_pose = 263
kinematic_chain = paramUtil.t2m_kinematic_chain

dim_word = 300
mean = np.load(pjoin(opt.data_root, 'Mean.npy'))
std = np.load(pjoin(opt.data_root, 'Std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')


train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, opt.times)

mock:: opt


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 2133.04it/s]


In [29]:
encoder = build_models(opt, dim_pose)

In [30]:
batch_size = 1 #opt.batch_size
x = torch.rand([batch_size, 40, 263])
timesteps = torch.rand([batch_size])
length = torch.ones([batch_size], dtype=int)
text = ['individual appears to be in a constant flrx.']*batch_size
_ = encoder(x, timesteps, length=length, text=text)

h torch.Size([1, 40, 512]) xf_out: torch.Size([1, 77, 256]) emb: torch.Size([1, 2048])


### Test Inference

In [3]:
opt.is_train = False
encoder = build_models(opt, dim_pose)
# encoder = MMDataParallel(
#             encoder.cuda(opt.gpu_id[0]), device_ids=opt.gpu_id)
trainer = DDPMTrainer(opt, encoder)
# trainer.load('../checkpoints/t2m/TEMP/model/latest.tar')
trainer.load('../checkpoints/t2m/t2m_motiondiffuse/model/latest.tar')
mean = np.load('../checkpoints/t2m/t2m_motiondiffuse/meta/mean.npy')
std = np.load('../checkpoints/t2m/t2m_motiondiffuse/meta/std.npy')
trainer.eval_mode()
trainer.to(opt.device)

In [6]:
with torch.no_grad():
    motion_length=120
    caption = ['A man walks forward.']
    m_lens = torch.LongTensor([motion_length]).to(opt.device)
    pred_motions, temp = trainer.generate(caption, m_lens, dim_pose, batch_size=1)
    motion = pred_motions[0].cpu().numpy()
#     motion = motion * std + mean

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]


In [10]:
temp[10][0].shape, pred_motions[0].shape

(torch.Size([120, 263]), torch.Size([120, 263]))

In [39]:
# _mean = mean.copy()
# motion_zero = np.zeros_like(motion)
# _mean[1:3] = 0 # xz velocity
# motion1 = motion_zero * std + _mean

motion1 = motion * std + mean

In [23]:
from mylib import animate3d
from utils.motion_process import recover_from_ric
motion1 = temp[990][0].cpu().numpy() * std + mean
joint = recover_from_ric(torch.from_numpy(motion1).float(), opt.joints_num).numpy()
animate3d(joint, save_path='990.html') # 'init.html'

### Train

In [12]:
del trainer
del encoder
torch.cuda.empty_cache()

encoder = build_models(opt, dim_pose)
encoder = MMDataParallel(
            encoder.cuda(opt.gpu_id[0]), device_ids=opt.gpu_id)
trainer = DDPMTrainer(opt, encoder)
trainer.train(train_dataset)

  0%|                                                                                                                                                                         | 0/50 [00:00<?, ?it/s]

num batch: 31
x: torch.Size([128, 196, 263]) timesteps:
timesteps: torch.Size([128])
length: torch.Size([128])
text: 512
xf: None
xf_out None
x: torch.Size([128, 196, 263]) timesteps:
timesteps: torch.Size([128])
length: torch.Size([128])
text: 512
xf: None
xf_out None
x: torch.Size([128, 196, 263]) timesteps:
timesteps: torch.Size([128])
length: torch.Size([128])
text: 512
xf: None
xf_out None
x: torch.Size([128, 196, 263]) timesteps:
timesteps: torch.Size([128])
length: torch.Size([128])
text: 512
xf: None
xf_out None


  0%|                                                                                                                                                                         | 0/50 [00:02<?, ?it/s]


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/epinyoan/miniconda3/envs/motiondiffuse/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/epinyoan/miniconda3/envs/motiondiffuse/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/epinyoan/git/MotionDiffuse/text2motion/study/../models/transformer.py", line 432, in forward
    h = module(h, xf_out, emb, src_mask)
  File "/home/epinyoan/miniconda3/envs/motiondiffuse/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/epinyoan/git/MotionDiffuse/text2motion/study/../models/transformer.py", line 193, in forward
    x = self.ffn(x, emb)
  File "/home/epinyoan/miniconda3/envs/motiondiffuse/lib/python3.10/site-p